In [20]:
import numpy as np
import gzip
import theano
from keras.models import load_model
import tensorflow as tf

In [21]:
def get_test():
    
    f = open('datasetTranskay/cb513+profile_split1.npy.gz', 'rb')
    cb513 = np.load(f)
    cb513 = np.reshape(cb513, (514, 700, 57))
    cb513 = cb513[:, :, :].astype(theano.config.floatX)

    return cb513

In [22]:
cb513 = get_test()

In [ ]:
for i in range(700):
    print(cb513[0,i,0:21])
    print(cb513[0,i,22:30])
    if(cb513[0,i,0:21].sum() == 0):
        print(i)
    if(cb513[0,i,22:30].sum() == 0):
        print(i)        

In [26]:
path = "model-assmebly-1.hdf5"
m = load_model( path )

In [27]:
from IPython.display import clear_output
import random

allValidWindows = []
all_targets = []

for i in range(cb513.shape[0]):
    
    print("protein nr " ,i)
    clear_output(wait=True)
    
    lastIndex = 700
    #ground truth
    for k in range(700):
        ground_truth = cb513[i,k,22:30]
        if(ground_truth.sum() == 0):
            lastIndex = k
            break
        all_targets.append(ground_truth)

    #creating padding version so we do not loose information
    padded = np.zeros((700+18, 57))
    padded[9:709] = cb513[i]
    
    for k in range(9):
        #padded[8-k] = cb513[i, lastIndex - 1 - k]
        padded[k] = cb513[i,0]#cb513[i,random.randint(0, lastIndex - 1)]#  #0# #
    
    nexta = 0
    for k in range(lastIndex + 9 , lastIndex + 9 + 9):
        #padded[k] = 0# cb513[i, 0 + nexta]
        nexta += 1
        padded[k] = 0#cb513[i,random.randint(0, lastIndex - 1)]#cb513[i,lastIndex - 1]##
        
    
    #making predictions
    predictions = []
    for k in range(718 - 19 + 1):

        if cb513[i,k,0:21].sum() == 0:
            break
        #make sure to check PSSM or 1 hot
        window =  padded[k:k+19, 35:56]
        allValidWindows.append(window)
    
scores = m.evaluate(np.array(allValidWindows), np.array(all_targets))
print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]))

84765/84765 [==============================] - 59s 701us/step
Loss: 0.8808585369941345, Accuracy: 0.7006547451019287, MAE: 0.09245184063911438


In [ ]:
for i in range(718):
  
    if i<9:
        print(padded[i,0:21])
    
    if i >= 9 and i <= 708:
        print(padded[i,0:21].all() == cb513[0,i-9,0:21].all())
    
    if i > 708:
        print(padded[i,0:21])

In [90]:
print(len(allValidWindows))

67


In [96]:
x = np.array(allValidWindows)

In [116]:
for i in range(19):
    print(x[0][i][0])

0.9950823758206397
0.06660803557509065
0.05315113639806372
0.061226162819907996
0.0316762283790564
0.4675456936126812
0.10065209356410687
0.14430313409051873
0.20424030228409176
0.2141650169574414
0.08317269649392238
0.34751053780725555
0.6615031592029524
0.960456220449047
0.35663485430559827
0.09195454993100434
0.43045377606077095
0.07585818002124355
0.08948005933356144


In [117]:
for i in range(19):
    print(x[66][i][0])

0.08948005933356144
0.9950823758206397
0.06660803557509065
0.05315113639806372
0.061226162819907996
0.0316762283790564
0.4675456936126812
0.10065209356410687
0.14430313409051873
0.20424030228409176
0.2141650169574414
0.08317269649392238
0.34751053780725555
0.6615031592029524
0.960456220449047
0.35663485430559827
0.09195454993100434
0.43045377606077095
0.07585818002124355
